# Tutorial 9: Combining LangChain and LangGraph

In this tutorial, we'll explore how to effectively combine LangChain components with LangGraph flows to create powerful, flexible AI applications. We'll build a sophisticated task planning and execution system that leverages the strengths of both libraries.

## Setup

First, let's import the necessary libraries and set up our environment:

In [ ]:
import os
from typing import Dict, TypedDict, List, Any
from langchain.llms import Groq
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseMessage
from langchain.agents import Tool
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END

# Set up the Groq LLM
llm = Groq(api_key=os.environ["GROQ_API_KEY"])

# Define our state structure
class State(TypedDict):
    messages: List[BaseMessage]
    current_step: str
    task_list: List[str]
    task_results: List[str]
    memory: Any

## 1. Integrating LangChain components into LangGraph flows

Let's start by creating some LangChain components that we'll integrate into our LangGraph flow:

In [ ]:
# Create a LangChain memory component
memory = ConversationBufferMemory(return_messages=True)

# Create LangChain tools
task_planner = Tool(
    name="TaskPlanner",
    func=lambda task: llm(f"Break down the following task into 3-5 subtasks: {task}").content,
    description="Breaks down a task into subtasks"
)

task_executor = Tool(
    name="TaskExecutor",
    func=lambda task: llm(f"Execute the following task and provide a brief result: {task}").content,
    description="Executes a given task and returns the result"
)

summarizer = Tool(
    name="Summarizer",
    func=lambda text: llm(f"Summarize the following text: {text}").content,
    description="Summarizes given text"
)

# Create LangChain chains
greeting_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template("Greet the user and ask what task they need help with today.")
)

## 2. Building a conversational AI system with both libraries

Now, let's create our LangGraph flow that incorporates these LangChain components:

In [ ]:
# Define our graph
workflow = StateGraph(State)

# Define our nodes
def greet_and_ask(state: State) -> State:
    response = greeting_chain.run()
    state["messages"].append({"role": "assistant", "content": response})
    state["current_step"] = "plan_task"
    return state

def plan_task(state: State) -> State:
    task = state["messages"][-1]["content"]
    subtasks = task_planner.run(task).split("\n")
    state["task_list"] = subtasks
    state["messages"].append({"role": "assistant", "content": f"I've broken down your task into the following subtasks:\n{' '.join(subtasks)}\nShall I proceed with execution?"})
    state["current_step"] = "get_user_confirmation"
    return state

def get_user_confirmation(state: State) -> str:
    response = state["messages"][-1]["content"].lower()
    if "yes" in response:
        return "execute_task"
    elif "no" in response:
        state["messages"].append({"role": "assistant", "content": "Understood. Is there anything else I can help you with?"})
        return "greet_and_ask"
    else:
        state["messages"].append({"role": "assistant", "content": "I didn't understand that. Please answer 'yes' to proceed or 'no' to start over."})
        return "get_user_confirmation"

def execute_task(state: State) -> State:
    if state["task_list"]:
        current_task = state["task_list"].pop(0)
        result = task_executor.run(current_task)
        state["task_results"].append(result)
        state["messages"].append({"role": "assistant", "content": f"Completed subtask: {current_task}\nResult: {result}"})
        state["current_step"] = "execute_task" if state["task_list"] else "summarize_results"
    else:
        state["current_step"] = "summarize_results"
    return state

def summarize_results(state: State) -> State:
    full_results = "\n".join(state["task_results"])
    summary = summarizer.run(full_results)
    state["messages"].append({"role": "assistant", "content": f"Task Summary:\n{summary}\nIs there anything else I can help you with?"})
    state["current_step"] = "check_for_more_tasks"
    return state

def check_for_more_tasks(state: State) -> str:
    response = state["messages"][-1]["content"].lower()
    if "yes" in response:
        return "greet_and_ask"
    elif "no" in response:
        state["messages"].append({"role": "assistant", "content": "Thank you for using our service. Have a great day!"})
        return END
    else:
        state["messages"].append({"role": "assistant", "content": "I didn't understand that. Please answer 'yes' if you need help with another task, or 'no' to end the conversation."})
        return "check_for_more_tasks"

# Add nodes to the graph
workflow.add_node("greet_and_ask", greet_and_ask)
workflow.add_node("plan_task", plan_task)
workflow.add_node("get_user_confirmation", get_user_confirmation)
workflow.add_node("execute_task", execute_task)
workflow.add_node("summarize_results", summarize_results)
workflow.add_node("check_for_more_tasks", check_for_more_tasks)

# Define edges
workflow.set_entry_point("greet_and_ask")
workflow.add_edge("greet_and_ask", "plan_task")
workflow.add_edge("plan_task", "get_user_confirmation")
workflow.add_edge("get_user_confirmation", "execute_task")
workflow.add_edge("get_user_confirmation", "greet_and_ask")
workflow.add_edge("execute_task", "execute_task")
workflow.add_edge("execute_task", "summarize_results")
workflow.add_edge("summarize_results", "check_for_more_tasks")
workflow.add_edge("check_for_more_tasks", "greet_and_ask")
workflow.add_edge("check_for_more_tasks", END)

# Compile the graph
app = workflow.compile()

## 3. Optimizing performance in complex applications

To optimize performance, we can implement caching for our LLM calls and use async operations where possible. Let's modify our setup to include these optimizations:

In [ ]:
from langchain.cache import InMemoryCache
from langchain.llms import Groq

# Set up caching
langchain.llm_cache = InMemoryCache()

# Use async LLM
llm_async = Groq(api_key=os.environ["GROQ_API_KEY"])

# Update tools to use async operations
task_planner = Tool(
    name="TaskPlanner",
    func=lambda task: llm_async.agenerate([f"Break down the following task into 3-5 subtasks: {task}"]),
    description="Breaks down a task into subtasks"
)

task_executor = Tool(
    name="TaskExecutor",
    func=lambda task: llm_async.agenerate([f"Execute the following task and provide a brief result: {task}"]),
    description="Executes a given task and returns the result"
)

summarizer = Tool(
    name="Summarizer",
    func=lambda text: llm_async.agenerate([f"Summarize the following text: {text}"]),
    description="Summarizes given text"
)

## 4. Case study: A task planning and execution system

Now that we have our optimized system set up, let's run it and see how it performs with a complex task:

In [ ]:
# Initialize the state
initial_state = {
    "messages": [],
    "current_step": "",
    "task_list": [],
    "task_results": [],
    "memory": memory
}

# Run the graph
async def run_workflow():
    async for output in app.astream(initial_state):
        if "messages" in output and output["messages"]:
            last_message = output["messages"][-1]
            if last_message["role"] == "assistant":
                print(f"Assistant: {last_message['content']}")
            elif last_message["role"] == "human":
                print(f"Human: {last_message['content']}")
        
        if output["current_step"] in ["plan_task", "get_user_confirmation", "check_for_more_tasks"]:
            user_input = input("Your response: ")
            output["messages"].append({"role": "human", "content": user_input})

    print("Workflow completed.")

# Run the workflow
import asyncio
asyncio.run(run_workflow())

Let's test our system with a complex task: "Plan a week-long vacation to Japan". Here's an example of how the interaction might go:

In [ ]:
# This is a simulated output. In practice, you would run the cell above and interact with the system.

print("Assistant: Hello! What task can I help you with today?")
print("Human: I need help planning a week-long vacation to Japan.")
print("Assistant: Certainly! I'd be happy to help you plan a week-long vacation to Japan. I'll break this task down into subtasks for us to work through. Here's what I propose:\n1. Research best times to visit Japan\n2. Identify top destinations and attractions\n3. Plan a day-by-day itinerary\n4. Research accommodation options\n5. Look into transportation within Japan\nShall I proceed with executing these subtasks?")
print("Human: Yes, please go ahead.")
print("Assistant: Great! I'll start working on these subtasks for you.")
print("Assistant: Completed subtask: Research best times to visit Japan\nResult: The best times to visit Japan are typically spring (March to May) for cherry blossoms and pleasant weather, and autumn (September to November) for colorful foliage and mild temperatures. Avoid the rainy season in June and the hot, humid summer months of July and August if possible.")
print("Assistant: Completed subtask: Identify top destinations and attractions\nResult: Top destinations in Japan include Tokyo (for modern culture, shopping, and technology), Kyoto (for traditional culture, temples, and gardens), Osaka (for food and nightlife), Hiroshima (for history), and Mount Fuji (for natural beauty). Key attractions: Tokyo Skytree, Senso-ji Temple, Fushimi Inari Shrine, Osaka Castle, Hiroshima Peace Memorial, and onsen (hot springs) experiences.")
print("Assistant: Completed subtask: Plan a day-by-day itinerary\nResult: Day 1-3: Tokyo (explore neighborhoods, visit temples, experience modern culture)\nDay 4-5: Kyoto (visit historic sites, enjoy traditional cuisine)\nDay 6: Day trip to Mount Fuji or Osaka\nDay 7: Return to Tokyo for departure, last-minute shopping and experiences")
print("Assistant: Completed subtask: Research accommodation options\nResult: Options range from traditional ryokans to modern hotels and budget-friendly hostels. In Tokyo, consider areas like Shinjuku or Shibuya for convenience. In Kyoto, staying in the Gion district offers a traditional atmosphere. Book in advance, especially during peak seasons.")
print("Assistant: Completed subtask: Look into transportation within Japan\nResult: The Japan Rail Pass is highly recommended for tourists, offering unlimited travel on JR trains, including most shinkansen (bullet trains). Within cities, subway systems are efficient. Consider renting a bicycle in Kyoto for local exploration. Taxis are available but can be expensive.")
print("Assistant: Task Summary:\nYour week-long vacation to Japan has been planned with a focus on experiencing both modern and traditional aspects of Japanese culture. The best times to visit are spring or autumn for optimal weather. Your itinerary includes 3 days in Tokyo, 2 days in Kyoto, a day trip to Mount Fuji or Osaka, and a final day in Tokyo. Key attractions have been identified in each location. Accommodation options range from ryokans to modern hotels, with recommendations for areas to stay in each city. For transportation, the Japan Rail Pass is recommended for inter-city travel, with subways and possibly bicycles for local transportation. Remember to book accommodations in advance, especially during peak seasons.\n\nIs there anything else I can help you with regarding your Japan trip planning?")
print("Human: No, that's all. Thank you!")
print("Assistant: Thank you for using our service. Have a great day!")
print("Workflow completed.")

## Conclusion

In this tutorial, we've demonstrated how to effectively combine LangChain components with LangGraph flows to create a sophisticated task planning and execution system. We've covered:

1. Integrating LangChain components (Tools, Chains, and Memory) into a LangGraph flow
2. Building a conversational AI system that can handle complex, multi-step tasks
3. Optimizing performance using caching and async operations
4. A case study of planning a week-long vacation to Japan

This combined approach allows for the creation of powerful, flexible AI applications that can handle a wide range of tasks. By leveraging the strengths of both LangChain and LangGraph, we can create systems that are both highly capable and easy to extend and maintain.

Some potential improvements and extensions to this system could include:
- Adding more specialized tools for different types of tasks
- Implementing more advanced error handling and fallback strategies
- Integrating external APIs for real-time data (e.g., weather, flight information)
- Expanding the memory capabilities to allow for longer-term context retention

As you continue to work with LangChain and LangGraph, experiment with different combinations of components and flow structures to find the best approach for your specific use cases.